In [2]:
import requests
import re
from bs4 import BeautifulSoup 
import bs4
import copy
import pandas as pd
from selenium import webdriver
import numpy as np

In [3]:
data8URL = "https://www.data8.org/sp23/staff/"
data100URL = "https://ds100.org/sp23/staff/"

In [4]:
data8page = requests.get(data8URL)

data8name = []
data8email = []

soup = BeautifulSoup(data8page.content, 'html.parser')
staffname = soup.find_all('h3')
email = soup.find_all(lambda tag: len(tag.find_all()) == 0 and "@berkeley.edu" in tag.text)

for line in staffname:
    data8name.append(line.text.strip())

for line in email:
    data8email.append(line.text.strip())
    
    

In [8]:
data100page = requests.get(data100URL)

data100name = []
data100name_dup = []
data100email = []
data100role = []

soup = BeautifulSoup(data100page.content, 'html.parser')
staffname = soup.find_all('h3')

#sort each line that includes staff name to extract only names (no pronouns) 
for line in staffname:
    #if there are no tag children, add to list
    if len(line.find_all()) == 0:
        data100name_dup.append(line.text.strip())
    
    for t in line.select('*'):
        #for names linked to website, sort through child tag p to exclude pronouns and find names 
        if t.text != 'She/Her/Hers' and t.text != 'He/Him/His' and t.text != 'They/Them/Theirs':
            data100name_dup.append(t.text)
        else: 
        #for tags that have children but are not linked to websites, add to list 
            t.extract()
            data100name_dup.append(line.get_text().strip())

for name in data100name_dup:
    if name not in data100name:
        data100name.append(name)

email = soup.find_all(lambda tag: len(tag.find_all()) == 0 and "@berkeley.edu" in tag.text)
for line in email:
    if line.text != '@huang33176@berkeley.edu': 
        data100email.append(line.text.strip())
data100email = data100email[1:]


role = soup.find_all('h2')
role
#use previous role 

[<h2 class="navigation-header">Links</h2>,
 <h2 id="course-staff-email">Course Staff Email</h2>,
 <h2 id="instructors">Instructors</h2>,
 <h2 id="lead-teaching-assistants">Lead Teaching Assistants</h2>,
 <h2 id="teaching-assistants">Teaching Assistants</h2>,
 <h2 id="readers">Readers</h2>]

In [10]:
data8dict = {'name': data8name, 'email': data8email, 'source': [data8URL for i in range(len(data8name))]}  
data100dict = {'name': data100name, 'email': data100email, 'source': [data100URL for i in range(len(data100name))]}
df8 = pd.DataFrame(data8dict) 
df100 = pd.DataFrame(data100dict) 
merged = pd.merge(df100, df8, how='outer')                                                                                       

merged

                                                                                            
                                                                                            
                                                                                                                                                                               

,name,email,source
0,Lisa Yan,yanlisa@berkeley.edu,https://ds100.org/sp23/staff/
1,Narges Norouzi,norouzi@berkeley.edu,https://ds100.org/sp23/staff/
2,Bella Crouch,isabella.crouch@berkeley.edu,https://ds100.org/sp23/staff/
3,Kanu Grover,grover.kanu@berkeley.edu,https://ds100.org/sp23/staff/
4,Liang Zhang,angelikazhang@berkeley.edu,https://ds100.org/sp23/staff/
...,...,...,...
122,Sarah Kiefer,sarahkiefer@berkeley.edu,https://www.data8.org/sp23/staff/
123,Sneha Mukherjee,sneham@berkeley.edu,https://www.data8.org/sp23/staff/
124,Sonya Shankar,sonyashankar@berkeley.edu,https://www.data8.org/sp23/staff/
125,Tiffany Liu,tiffany.yt.liu@berkeley.edu,https://www.data8.org/sp23/staff/
